In [54]:
import pandas as pd
data = pd.read_csv("filtered_solved_ac_data.csv")


In [13]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# 태그 리스트
tag_list = [
    "브루트포스 알고리즘",
    "구현",
    "수학",
    "사칙연산",
    "그리디 알고리즘",
    "이분 탐색",
    "기하학",
    "그래프 이론",
    "그래프 탐색",
    "조합론",
    "너비 우선 탐색",
    "다이나믹 프로그래밍",
    "백트래킹",
    "자료 구조",
    "깊이 우선 탐색",
    "시뮬레이션",
    "문자열",
    "정수론",
    "트리"
]


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 4)


def get_algorithm_tags(user_name):
    url = f"https://solved.ac/profile/{user_name}"
    driver.get(url)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    # "더 보기" 버튼을 두 번 클릭
    try:
        for _ in range(2):
            driver.execute_script("""
            var buttons = document.querySelectorAll('button');
            for (var i = 0; i < buttons.length; i++) {
                if (buttons[i].innerText.includes('더 보기')) {
                    buttons[i].click();
                    break;
                }
            }
            """)
            print("JavaScript로 '더 보기' 버튼을 클릭했습니다.")
            time.sleep(0.2)


    except Exception as e:
        print(f"더 보기 버튼 클릭 오류: {e}")


    # 다시 페이지를 끝까지 스크롤합니다.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    tags_ratings_problems = {}


    # 각 태그와 레이팅 및 푼 문제 크롤링
    try:
        # 태그 테이블의 모든 행을 찾습니다.
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        print(f"태그 테이블에서 {len(rows)}개의 행을 찾았습니다.")


        for index, row in enumerate(rows):
            try:
                # 행의 모든 셀(td 요소들)을 가져옵니다.
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) < 4:
                    continue  # 셀이 부족하면 건너뜁니다.


                # 태그명 추출 (첫 번째 td의 a 태그)
                tag_name_element = cells[0].find_element(By.TAG_NAME, "a")
                tag_name = tag_name_element.text.strip()
        
                tag_name_cleaned = tag_name.replace('#', '').lower()

                # 푼 문제 수 추출 (두 번째 td)
                problems_solved = cells[1].text.strip()


                # 티어와 레이팅 추출 (네 번째 td에 있는 img 태그의 alt 속성)
                rating_td = cells[3]
                tier_img = rating_td.find_element(By.TAG_NAME, "img")
                tier = tier_img.get_attribute("alt").strip()
               
                # 레이팅 숫자만 추출
                rating_text = rating_td.text.strip()
                rating_value_match = re.search(r'(\d+)', rating_text)
                rating_value = rating_value_match.group(1) if rating_value_match else "0"
               
                # 티어와 레이팅 결합
                combined_rating = f"{tier}, {rating_value}"


                # 미리 정의된 태그 리스트에 있는 경우에만 추가
                if tag_name_cleaned in tag_list:
                    tags_ratings_problems[f"{tag_name_cleaned}_rating"] = combined_rating
                    tags_ratings_problems[f"{tag_name_cleaned}_problems_solved"] = problems_solved
                    print(f"{index + 1}번째 행 - 태그: {tag_name_cleaned}, 티어와 레이팅: {combined_rating}")
            except Exception as e:
                print(f"{index + 1}번째 행 파싱 중 오류: {e}")


    except Exception as e:
        print(f"알고리즘 태그 및 티어 수집 오류: {e}")
        # 태그 테이블이 없을 경우 모든 태그에 대해 "Unrated"으로 초기화
        for tag in tag_list:
            tags_ratings_problems[f"{tag}_rating"] = "Unrated"
            tags_ratings_problems[f"{tag}_problems_solved"] = "0"
        return tags_ratings_problems


    # 모든 태그에 대해 값이 없으면 "Unrated"으로 설정
    for tag in tag_list:
        if f"{tag}_rating" not in tags_ratings_problems:
            tags_ratings_problems[f"{tag}_rating"] = "Unrated"
            tags_ratings_problems[f"{tag}_problems_solved"] = "0"


    return tags_ratings_problems


# 최종 데이터를 담을 리스트
all_data = []


def fetch_user_data(user_name, user_tier, user_solved_count, filtered_users):
    try:
        # 1단계: 알고리즘 태그와 레이팅 및 푼 문제 수 수집
        tags_ratings_problems = get_algorithm_tags(user_name)
        print(f"사용자 {user_name}의 태그 레이팅: {tags_ratings_problems}")
        user_tag_data = {f"{tag} 레이팅": tags_ratings_problems.get(f"{tag}_rating", "Unrated") for tag in tag_list}
        user_tag_data.update({f"{tag} 푼 문제 수": tags_ratings_problems.get(f"{tag}_problems_solved", "0") for tag in tag_list})


        # 2단계: 유저의 기본 정보와 문제 데이터 합치기
        user_problem_data = filtered_users[filtered_users['닉네임'] == user_name]


        for _, problem_row in user_problem_data.iterrows():
            user_data = {
                "티어": user_tier,
                "닉네임": user_name,
                "푼 문제 수": user_solved_count,
                **user_tag_data,
                "문제 번호": problem_row['문제 번호'],
                "문제 티어": problem_row['문제 티어'],
                "문제 이름": problem_row['문제 이름'],
                "문제 푼 사람 수": problem_row['문제 푼 사람 수'],
                "문제 태그": problem_row['문제 태그'],
                "문제 링크": problem_row['문제 링크'],
            }


            all_data.append(user_data)


        print(f"사용자 {user_name}의 데이터 수집 완료.")


        return user_name


    except Exception as e:
        print(f"에러 발생: {e}")
        return None


def save_data():
    df = pd.DataFrame(all_data)
   
    # 기본 열
    columns_order = ['티어', '닉네임', '푼 문제 수']
   
    # 태그별 레이팅과 푼 문제 수를 짝으로 정렬
    ratings_problems_columns = []
    for tag in tag_list:
        ratings_problems_columns.append(f"{tag} 레이팅")
        ratings_problems_columns.append(f"{tag} 푼 문제 수")
   
    # 문제 관련 열
    problem_columns = ['문제 번호', '문제 티어', '문제 이름', '문제 푼 사람 수', '문제 태그', '문제 링크']
   
    # 최종 열 순서로 데이터프레임을 재정렬
    df = df[columns_order + ratings_problems_columns + problem_columns]
   
    # CSV 파일 저장(다만 좀 번거로운게 티어를 Gold II면 Gold II 로 바꿔서 저장 필요.)
    df.to_csv("Platinum I_users_solved_ac_data.csv", index=False, encoding="utf-8-sig")
    print("중간 데이터 저장 완료.")


# 필터링된 유저 목록에 대해 순차적으로 수집 작업 실행
try:
    data = pd.read_csv("filtered_solved_ac_data.csv")
    # 마찬가지로 Gold II 면 Gold II 로 바꿔서.
    filtered_users = data[data['티어'] == 'Platinum I'][['티어', '닉네임', '푼 문제 수', '문제 번호', '문제 티어', '문제 이름', '문제 푼 사람 수', '문제 태그', '문제 링크']]


    # 유저 데이터를 순차적으로 수집
    for idx, row in filtered_users.drop_duplicates(subset='닉네임').iterrows():
        user_tier = row['티어']
        user_name = row['닉네임']
        user_solved_count = row['푼 문제 수']
        fetch_user_data(user_name, user_tier, user_solved_count, filtered_users)


except Exception as e:
    print(f"에러 발생: {e}")


finally:
    # 중간 또는 최종 데이터 저장
    save_data()
    driver.quit()



C:\Users\selah\AppData\Local\Temp\ipykernel_13640\2491726335.py:237: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("filtered_solved_ac_data(Gold II ~ Platinum I).csv")


JavaScript로 '더 보기' 버튼을 클릭했습니다.
JavaScript로 '더 보기' 버튼을 클릭했습니다.
태그 테이블에서 36개의 행을 찾았습니다.
7번째 행 - 태그: 자료 구조, 티어와 레이팅: Platinum IV, 1779
8번째 행 - 태그: 그래프 이론, 티어와 레이팅: Platinum V, 1729
9번째 행 - 태그: 구현, 티어와 레이팅: Gold I, 1558
10번째 행 - 태그: 그래프 탐색, 티어와 레이팅: Gold I, 1479
11번째 행 - 태그: 문자열, 티어와 레이팅: Gold II, 1347
12번째 행 - 태그: 다이나믹 프로그래밍, 티어와 레이팅: Gold II, 1338
13번째 행 - 태그: 브루트포스 알고리즘, 티어와 레이팅: Gold II, 1304
14번째 행 - 태그: 백트래킹, 티어와 레이팅: Gold II, 1269
15번째 행 - 태그: 수학, 티어와 레이팅: Gold III, 1176
17번째 행 - 태그: 너비 우선 탐색, 티어와 레이팅: Gold IV, 1086
18번째 행 - 태그: 시뮬레이션, 티어와 레이팅: Gold IV, 1070
19번째 행 - 태그: 이분 탐색, 티어와 레이팅: Gold V, 838
21번째 행 - 태그: 트리, 티어와 레이팅: Silver I, 662
22번째 행 - 태그: 깊이 우선 탐색, 티어와 레이팅: Silver I, 655
24번째 행 - 태그: 그리디 알고리즘, 티어와 레이팅: Silver III, 470
27번째 행 - 태그: 정수론, 티어와 레이팅: Silver IV, 369
32번째 행 - 태그: 조합론, 티어와 레이팅: Silver V, 261
33번째 행 - 태그: 기하학, 티어와 레이팅: Silver V, 243
사용자 cksgur1765의 태그 레이팅: {'자료 구조_rating': 'Platinum IV, 1779', '자료 구조_problems_solved': '122', '그래프 이론_rating': 'Platinum V, 1729', '그